# 4.2: Sort missed data streams
Notebook 4.1 created text files containing the reference designator and data stream pairs that were not added to the missed test CSVs. These data streams either didn't have a matching test in the Gross Range and Climatology Test lookup tables, or the data for the first available deployment could not be loaded into the workspace. 
This notebook completes the following tasks:
    <ul><li>Sort strings of refdes-datastream pairs into one of two lists based on the issue printed to output</li>
    </ul>

In [1]:
# Import libraries
import re

In [2]:
def find_no_dataset(path_to_file):
    # Lists all data streams where data from the first
    # available deployment did not load in notebook
    # 4.1.
    file = open(path_to_file, "r")
    all_missing = list()
    for text in file.readlines():
        if "No dataset available" in text:
            missing = re.findall(".{8}-.{5}-.{2}-.{9}-[a-z_0-9]+", text)
            all_missing.append(missing)
    return all_missing

def find_no_qc_lookup(path_to_file):
    # Lists all data streams where no qcConfig objects
    # remained after filtering in notebook 4.1.
    file = open(path_to_file, "r")
    all_missing = list()
    for text in file.readlines():
        if "No existing qc-lookup table" in text:
            missing = re.findall(".{8}-.{5}-.{2}-.{9}-[a-z_0-9]+", text)
            all_missing.append(missing)
    return all_missing

def filter_streams(missing_qcConfig):
    # Remove streams where an automated
    # QC test is not expected according
    # to keywords in the stream name.
    drop = ["metadata", "diagnostic", "blank"]
    drop_mask = missing_qcConfig in drop
    missing_qcConfig = missing_qcConfig[drop_mask]
    return missing_qcConfig

In [39]:
array = "CP0"
site = "CP05MOAS"
path = f"../data/processed/{array}_tests_completed/{site}-typescript.txt"

In [40]:
no_deploy1 = find_no_dataset(path)
no_deploy1

[['CP05MOAS-PG564-01-CTDGVM000-ctdgv_m_glider_instrument_recovered'],
 ['CP05MOAS-PG564-01-CTDGVM000-ctdgv_m_glider_instrument'],
 ['CP05MOAS-A6263-03-CTDAVN000-ctdav_auv_data']]

In [41]:
no_qcConfig = find_no_qc_lookup(path)
no_qcConfig

[['CP05MOAS-PG583-06-PARADM000-parad_m_glider_recovered'],
 ['CP05MOAS-PG583-06-PARADM000-parad_m_glider_instrument'],
 ['CP05MOAS-PG583-05-NUTNRM000-nutnr_m_glider_instrument'],
 ['CP05MOAS-PG583-05-NUTNRM000-nutnr_m_glider_instrument'],
 ['CP05MOAS-PG583-04-FLORTO000-flort_o_glider_data'],
 ['CP05MOAS-PG583-04-FLORTO000-flort_o_glider_data'],
 ['CP05MOAS-PG583-03-FLORTM000-flort_m_sample'],
 ['CP05MOAS-PG583-03-FLORTM000-flort_m_sample'],
 ['CP05MOAS-PG583-02-DOSTAM000-dosta_abcdjm_glider_recovered'],
 ['CP05MOAS-PG583-02-DOSTAM000-dosta_abcdjm_glider_instrument'],
 ['CP05MOAS-PG583-00-ENG000000-glider_eng_metadata_recovered'],
 ['CP05MOAS-PG583-00-ENG000000-glider_eng_recovered'],
 ['CP05MOAS-PG583-00-ENG000000-glider_eng_sci_recovered'],
 ['CP05MOAS-PG583-00-ENG000000-glider_gps_position'],
 ['CP05MOAS-PG583-00-ENG000000-glider_eng_metadata'],
 ['CP05MOAS-PG583-00-ENG000000-glider_eng_telemetered'],
 ['CP05MOAS-PG583-00-ENG000000-glider_gps_position'],
 ['CP05MOAS-PG564-06-PARADM00